# Getting Data

In [25]:
def get_stock_info(ticker_symbol="AAPL", period='10y'):
    import yfinance as yf
    import pandas as pd
    stock_data = yf.download(ticker_symbol, period=period)
    stock_data.reset_index(inplace=True)
    return stock_data

In [26]:
stock_data = get_stock_info()

[*********************100%***********************]  1 of 1 completed


# Converting Prices to Returns

In [15]:
def get_returns(stock_data, colname = "Adj Close"):
    # Generate returns for the column specified from ("Open", "High", "Low", "Close", "Adj Close")
    
    stock_data["Return"] = stock_data["Adj Close"].pct_change()
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    return stock_data[["Date", "Return"]].dropna()

In [16]:
returns = get_returns(stock_data)
print(returns.head())

        Date    Return
1 2014-07-15 -0.011716
2 2014-07-16 -0.005665
3 2014-07-17 -0.017831
4 2014-07-18  0.014395
5 2014-07-21 -0.005189


# Simple Return Formulas

In [38]:
def simple_return_k(old_date, new_date, stock_data, colname = "Adj Close"):
    # Ensure the dates are in datetime format
    old_date = pd.to_datetime(old_date)
    new_date = pd.to_datetime(new_date)
    
    # Filter the stock data for the given dates
    old_price = stock_data.loc[stock_data['Date'] == old_date, colname].values[0]
    new_price = stock_data.loc[stock_data['Date'] == new_date, colname].values[0]
    
    # Calculate the simple return
    r_t = (new_price - old_price) / old_price
    print(f"P_t={old_price}, P_t{(old_date-new_date).days} = {new_price}, 1+R_t={r_t+1}, R_t = {r_t}")
    return r_t
def simple_return_one(date, stock_data, colname = "Adj Close"):
    date = pd.to_datetime(date)
    curr_price = stock_data.loc[stock_data["Date"] == date, colname].values[0]
    yest_price = stock_data.loc[stock_data["Date"] == date- pd.Timedelta(days=1), colname].values[0]
    r_t = (curr_price - yest_price) / yest_price
    print(f"P_t = {curr_price}, P_t-1 = {yest_price}, 1+R_t = {r_t+1}, R_t = {r_t}")
    return r_t

In [39]:
simple_return_k("2014-07-14", "2014-07-17", stock_data)
simple_return_one("2014-07-16", stock_data)
returns.head()

P_t=21.328353881835938, P_t-3 = 20.585342407226562, 1+R_t=0.9651632058092325, R_t = -0.03483679419076747
P_t = 20.959062576293945, P_t-1 = 21.07847785949707, 1+R_t = 0.9943347292912177, R_t = -0.005665270708782301


,Date,Return
1,2014-07-15,-0.011716
2,2014-07-16,-0.005665
3,2014-07-17,-0.017831
4,2014-07-18,0.014395
5,2014-07-21,-0.005189


# Multiperiod Return Formulas

In [46]:
def compound_return_k(old_date, new_date, stock_data, colname = "Adj Close"):
    # Ensure the dates are in datetime format
    old_date = pd.to_datetime(old_date)
    new_date = pd.to_datetime(new_date)
    return_array = []
    curr_date = new_date
    days = 0
    while curr_date > old_date:
        curr_price = stock_data.loc[stock_data['Date'] == curr_date, colname].values[0]
        yest_price = stock_data.loc[stock_data['Date'] == curr_date- pd.Timedelta(days=1), colname].values[0]
        r_t = (curr_price - yest_price) / yest_price
        print(f"P_t-{(new_date-curr_date).days} = {curr_price}, P_t-{(new_date-curr_date).days+1} = {yest_price}, 1+R_t = {r_t+1}, R_t = {r_t}")
        return_array.append(1+r_t)
        curr_date = curr_date- pd.Timedelta(days=1)
        days += 1
    print("List of Returns (1+R_t)", return_array)
    import numpy as np
    print("Multiperiod Compound Return:", np.prod(return_array))
    return np.prod(return_array)

In [47]:
compound_return_k("2014-07-14", "2014-07-17", stock_data)

P_t-0 = 20.585342407226562, P_t-1 = 20.959062576293945, 1+R_t = 0.9821690417829048, R_t = -0.017830958217095287
P_t-1 = 20.959062576293945, P_t-2 = 21.07847785949707, 1+R_t = 0.9943347292912177, R_t = -0.005665270708782301
P_t-2 = 21.07847785949707, P_t-3 = 21.328353881835938, 1+R_t = 0.9882843268766432, R_t = -0.011715673123356763
List of Returns (1+R_t) [0.9821690417829048, 0.9943347292912177, 0.9882843268766432]
Multiperiod Compound Return: 0.9651632058092326


0.9651632058092326